In [1]:
from MNE_Pipeline_ext import MNE_Repo_Mat_ext
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pickle

In [2]:
mp = MNE_Repo_Mat_ext()

In [3]:
def arrange_event_wise_RTs(RT, trigs):
    RT_dict = dict()
    event_ids = np.unique(trigs)
    for event in event_ids:
        indices = [i for i in range(len(trigs)) if trigs[i] == event]
        RT_dict[str(event)] = RT[indices]
        
    return RT_dict

In [4]:
subjects = sorted([os.path.splitext(file)[0] for file in os.listdir('Data/') if file.endswith('.mat')])
subjects_path = sorted(['Data/{}'.format(file) for file in os.listdir('Data/') if file.endswith('.mat')])
subject_epoch_paths = [os.path.join('combined_epochs', subject + '_epoch' + '.fif') for subject in subjects]

In [28]:
RT_dict_sub = dict()
for subject, subject_path in tqdm(zip(subjects, subjects_path)):
    data = mp.load_data(subject_path)
    RT = np.nan_to_num(data['RT'], nan=0)
    trigs = data['trigs']
    trigs = []
    RT_dict_sub[subject] = arrange_event_wise_RTs(RT, trigs)

50it [00:59,  1.18s/it]


In [30]:
with open('RT_dict_sub', 'wb') as f:
    pickle.dump(RT_dict_sub, f)

In [5]:
for path, subject in tqdm(zip(subject_epoch_paths, subjects)):
    epochs = mp.load_epochs(path)
    mp.async_bootstrap_epochs(subject, epochs, iterations=500)

0it [00:00, ?it/s]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This filename (combined_epochs/N1_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
1it [00:06,  6.78s/it]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This filename (combined_epochs/N10_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
2it [00:13,  6.79s/it]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This filename (combined_epochs/N11_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
3it [00:20,  6.78s/it]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This filename (c

28it [03:10,  6.87s/it]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This filename (combined_epochs/S21_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
29it [03:17,  6.88s/it]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This filename (combined_epochs/S3_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
30it [03:24,  6.88s/it]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This filename (combined_epochs/S4_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
31it [03:31,  6.88s/it]/home/kashraf/cp_project/MNE_Pipeline.py:76: RuntimeWarning: This file

In [10]:
with open('bootstrap_erps_cl_vs_amb/T9', 'rb') as file:
    di = pickle.load(file)

In [11]:
di['3'].shape

(500, 64, 500)

In [6]:
correction_paths = [path for path in subject_epoch_paths if 'N2_' in path or 'N12_' in path or 'T4_' in path or 'T6_' in path]
correction_subjects = [subject for subject in subjects if subject == 'N2' or subject == 'N12' or subject == 'T4' or subject == 'T6']

In [7]:
for path, subject in tqdm(zip(correction_paths, correction_subjects)):
    epochs = mp.load_epochs(path)
    mp.async_bootstrap_epochs(subject, epochs)

0it [00:00, ?it/s]/home/kam/CP Project/MNE_Pipeline.py:76: RuntimeWarning: This filename (epochs/N12_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
1it [00:08,  8.34s/it]/home/kam/CP Project/MNE_Pipeline.py:76: RuntimeWarning: This filename (epochs/N2_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
2it [00:16,  8.33s/it]/home/kam/CP Project/MNE_Pipeline.py:76: RuntimeWarning: This filename (epochs/T4_epoch.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
3it [00:24,  8.33s/it]/home/kam/CP Project/MNE_Pipeline.py:76: RuntimeWarning: This filename (epochs/T6_epoch.fif) does not conform to MNE 

In [12]:
bts_sample_path = [os.path.join('bootstrap_erps', subject) for subject in subjects]

In [14]:
df = pd.DataFrame(columns=['subject', 'trig', 'indices'])

In [ ]:
for path